In [ ]:
import os
from google import genai
from google.genai import types
from IPython.display import Markdown, display
from google.api_core import retry

# Get API Key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = input("Please enter your Google API key: ")
if not GOOGLE_API_KEY:
    raise ValueError("Google API key not found. Please set the GOOGLE_API_KEY environment variable or provide it manually.")

# Initialize GenAI client
client = genai.Client(api_key=GOOGLE_API_KEY)


In [ ]:
# Retry policy for quota or service limit errors
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

# Apply retry only once to avoid multiple wrappings
if not hasattr(genai.models.Models.generate_content, '_wrapped_'):
    genai.models.Models.generate_content = retry.Retry(predicate=is_retriable)(
        genai.models.Models.generate_content
    )


In [ ]:
# Enable grounding via Google Search
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)


In [ ]:
# Chat memory (list of alternating user and model messages)
conversation_history = []


In [ ]:
print("Chatbot with memory and search grounding started! Type 'quit' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        print("Exiting the chatbot. Goodbye!")
        break

    # Add user message to memory with correct Part format
    conversation_history.append(types.Content(role="user", parts=[types.Part(text=user_input)]))

    try:
        # Send full conversation context to model
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=conversation_history,
            config=config_with_search,
        )

        # Get and display response
        reply_text = response.candidates[0].content.parts[0].text
        display(Markdown(reply_text))

        # Add model reply to memory correctly
        conversation_history.append(types.Content(role="model", parts=[types.Part(text=reply_text)]))

    except Exception as e:
        print("An error occurred:", e)
